In [1]:
from transformers import MBartForConditionalGeneration

In [2]:
from huggingface_hub import notebook_login
from transformers import MBart50Tokenizer


In [13]:
tokenizer = MBart50Tokenizer.from_pretrained("/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/tok",max_len=256)
model = MBartForConditionalGeneration.from_pretrained("/mnt/disk/yrajcoomar/kreol-benchmark/checkpoint/checkpoint-72000")

In [14]:
example_english_phrase = "I will not follow you wherever you go"

inputs = tokenizer(example_english_phrase, return_tensors="pt")

generated_tokens = model.generate(**inputs)

tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


['Mo pa pou lev twa kot to ete?']

In [15]:
import evaluate

bleu = evaluate.load("bleu")

In [16]:
import pandas as pd
val = pd.read_json('/mnt/disk/yrajcoomar/kreol-benchmark/experiments/data/en-cr/en-cr_dev.jsonl',lines=True)

In [17]:
val_inputs = list(val['input'])
val_labels = list(val['target'])

In [18]:
input_tokens = tokenizer(val_inputs,max_length=128, truncation=True, padding="max_length",return_tensors='pt')

In [19]:
output_tokens = model.generate(**input_tokens)
output = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)

/home/ubuntu/miniconda3/envs/cuda11/lib/python3.10/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [20]:
bleu.compute(predictions=output,references=val_labels)

{'bleu': 0.04176882522734308,
 'precisions': [0.4168018164125851,
  0.11048358630427109,
  0.03871467286101433,
  0.013056506849315069],
 'brevity_penalty': 0.6013395530687732,
 'length_ratio': 0.6628682003870136,
 'translation_length': 6166,
 'reference_length': 9302}

In [12]:
for i in range(10):
    print(f"Input: {val_inputs[i]}")
    print(f"Prediction: {output[i]}")
    print(f"Label: {val_labels[i]}")
    print('------------')


Input: I did not come to do away with them, but to give them their full meaning.
Prediction: Mo pa ti vini avek zot, me zot ti fer zot per zot.
Label: Mo pa finn vini pou aboli me pou donn zot zot vre sinifikasion.
------------
Input: The fact is, at the time, you had to pay the teacher in order to go to school.
Prediction: Letan zot al lekol, zot ti fer nwaye dan lekol.
Label: Anverite sa lepok la pou al lekol ti ena enn fiz pou pey profeser.
------------
Input: Angina can be described as a discomfort, heaviness, pressure, aching, burning.
Prediction: Antrefwa “mili” enn antropolog, dibwa, par lafors gravite.
Label: Nou capav dekrir anzinn couma enn sensasion inkonfortab, lourder, presion.
------------
Input: The boy said he would, but he didn't go.
Prediction: Bann garson la ti pou al plis, me li pa ti ale.
Label: Garson-la reponn wi papa, li pou ale me li pa ale.
------------
Input: Was it God in heaven or merely some human being?
Prediction: Eski u ti dan enn lot ka?
Label: Eski sa

In [33]:
val_inputs[0]

'I did not come to do away with them, but to give them their full meaning.'

In [34]:
val_labels[0]

'Mo pa finn vini pou aboli me pou donn zot zot vre sinifikasion.'

In [32]:
tokenizer.batch_decode(output, skip_special_tokens=True)


['Mo pa vinn zwenn zot, me zot ti pe dimann zot.']

In [31]:
output

tensor([[    0, 20004,   890,   101,   230,  1196,    81, 19941,   190,    81,
            32,    68,  1104,    81, 19940,     2]])

In [5]:
val

,input,target
0,"I did not come to do away with them, but to gi...",Mo pa finn vini pou aboli me pou donn zot zot ...
1,"The fact is, at the time, you had to pay the t...",Anverite sa lepok la pou al lekol ti ena enn f...
2,"Angina can be described as a discomfort, heavi...",Nou capav dekrir anzinn couma enn sensasion in...
3,"The boy said he would, but he didn't go.","Garson-la reponn wi papa, li pou ale me li pa ..."
4,Was it God in heaven or merely some human being?,Eski sa ti sorti depi dan lesiel ouswa dimoun ...
...,...,...
495,"The angel answered, the Holy Spirit will come ...","Anz la reponn, Lespri Sin pou vinn lor twa, e ..."
496,"At its end, it had a whole lot of Flame-Trees ...","Dan so finision, de kote sime, ti ena enn ta p..."
497,"The king will answer, whenever you did it for ...","Lerla lerwa reponn zot, sak fwa ki zot finn fe..."
498,You Pharisees and teachers of the Law of Moses...,"Maler lor zot profeser lalwa Moiz ek Farizien,..."


In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}